In [ ]:
# pip install pymupdf sentence-transformers chromadb pillow

  Using cached pillow-12.1.0-cp310-cp310-win_amd64.whl.metadata (9.0 kB)
  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached markupsafe-3.0.3-cp310-cp310-win_amd64.whl.metadata (2.8 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/18.4 MB ? eta -:--:--
   ------------ --------------------------- 5.8/18.4 MB 35.3 MB/s eta 0:00:01
   ------------------------------ --------- 14.2/18.4 MB 37.0 MB/s eta 0:00:01
   ---------------------------------------- 18.4/18.4 MB 35.2 MB/s  0:00:00
   ---------------------------------------- 0.0/10.1 MB ? eta -:-

In [3]:
client

In [1]:
import os
from typing import List , Dict , Any

import fitz

from PIL import Image
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb 

d:\Work\learning\GEN-ai\vectordb_VENV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
PDF_PATH = r"data\1706.03762v7.pdf"
FIGURE_DIR = r"data\figure"

os.makedirs(FIGURE_DIR , exist_ok=True)


In [3]:
import chromadb

client = chromadb.CloudClient(
  api_key='ck-GPKiqKqGR2YzvTqw6kxYwAU1tccp5zLEmRGFBhkAc5Yf',
  tenant='008c8583-0f4f-4a4e-ac8a-f40bfc731074',
  database='imageandtext'
)

In [4]:
FIG_COLLECTION_NAME = "pdf_figure"
TEXT_COLLECTION_NAME = "attention_text_chunks"

In [ ]:
model = SentenceTransformer('clip-ViT-B-32') # dimention of model is 512

d:\Work\learning\GEN-ai\vectordb_VENV\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sagnik\.cache\huggingface\hub\models--sentence-transformers--clip-ViT-B-32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 398/398 [00:00<00:00, 872.91it/s, Materializing param=visual

In [6]:
def encode_image(path:str)->np.ndarray:
    img = Image.open(path).convert("RGB")
    emb = model.encode([img] , convert_to_numpy=True , show_progress_bar=False)
    return emb[0]

def encode_text(text:str)->np.ndarray:
    
    emb = model.encode([text] , convert_to_numpy=True , show_progress_bar=False)
    return emb[0]

In [9]:
doc = fitz.open(PDF_PATH)

In [10]:
doc.metadata

{'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.25',
 'creationDate': 'D:20240410211143Z',
 'modDate': 'D:20240410211143Z',
 'trapped': '',
 'encryption': None}

In [11]:
# for image will crete one index and for text one index

In [13]:
image_records: List[Dict[str, Any]] = []

print("Extracting images and captions...")
for page_index, page in enumerate(doc, start=1):
    blocks = page.get_text("blocks")  # (x0, y0, x1, y1, text, ...)
    text_blocks = [b for b in blocks if b[4].strip()]  # keep non-empty text blocks

    images = page.get_images(full=True)
    print(f"Page {page_index}: found {len(images)} images")

    for img_index, img in enumerate(images, start=1):
        xref = img[0]

        # Where is this image on the page?
        rects = page.get_image_rects(xref)
        if not rects:
            continue
        rect = rects[0]

        # Heuristic: caption = closest text block BELOW the image
        below_blocks = [b for b in text_blocks if b[1] >= rect.y1]
        candidates = below_blocks if below_blocks else text_blocks

        if candidates:
            closest_block = min(candidates, key=lambda b: abs(b[1] - rect.y1))
            caption_text = closest_block[4].strip().replace("\n", " ")
        else:
            caption_text = ""

        # Extract image binary and save to file
        img_data = doc.extract_image(xref)
        img_bytes = img_data["image"]
        img_ext = img_data.get("ext", "png")

        img_filename = f"page_{page_index:02d}_img_{img_index}.{img_ext}"
        img_path = os.path.join(FIGURE_DIR, img_filename)

        with open(img_path, "wb") as f:
            f.write(img_bytes)

        image_records.append({
            "id": f"p{page_index}_img{img_index}",
            "page_number": page_index,
            "bbox_x0": float(rect.x0),
            "bbox_y0": float(rect.y0),
            "bbox_x1": float(rect.x1),
            "bbox_y1": float(rect.y1),
            "caption": caption_text,
            "image_path": img_path,
        })

print(f"Total extracted images: {len(image_records)}")

Extracting images and captions...
Page 1: found 0 images
Page 2: found 0 images
Page 3: found 1 images
Page 4: found 2 images
Page 5: found 0 images
Page 6: found 0 images
Page 7: found 0 images
Page 8: found 0 images
Page 9: found 0 images
Page 10: found 0 images
Page 11: found 0 images
Page 12: found 0 images
Page 13: found 0 images
Page 14: found 0 images
Page 15: found 0 images
Total extracted images: 3
